In [ ]:
import ipywidgets as widgets
from ipywidgets import Label, Layout, HBox, HTML
import IPython
import IPython.display as display
from ipyevents import Event
import pathlib
import pycdlib
import fnmatch
from IPython.core.display import HTML
from path import Path
from urllib.request import urlopen
from ipywidgets import GridspecLayout
import math
import os
import re

In [ ]:
!jupyter serverextension enable --user voila ipyevents

In [ ]:
pattern = ["ALCH*","CPCS*","GUST*","PAPX*","PBGP*","PBPX*","PCPX*","PDPX*",
           "PKP2*","SCAJ*","SCCS*","SCED*","SCES*","SCKA*","SCPM*","SCPN*","SCPS*",
           "SCUS*","SLAJ*","SLED*","SLES*","SLKA*","SLPM*","SLPS*","SLUS*","TCES*",
           "TCPS*","TLES*", "VW067*"]

iso = pycdlib.PyCdlib()
gamelist = list(pathlib.Path(os.getcwd()).glob('*.iso'))
id_dictionary = {}

for x in range (len(gamelist)):
    iso.open(str(gamelist[x]).replace(os.getcwd() + '\\',''))
    for child in iso.list_children(iso_path='/'):
        for index in range(len(pattern)):
            if fnmatch.fnmatch(child.file_identifier().decode("utf-8"), pattern[index]):
                id_dictionary["ID" + str(x)] = child.file_identifier().decode("utf-8").replace('_', '-').replace('.', '').replace(';1', '')
                iso.close()

In [ ]:
%%html

<style>

*, ::before, ::after {
    max-height: 530px;
    flex-wrap: wrap;
}

.widget-html{
    position: absolute;
}

.widget-html-content{
    display: inherit;
}

.jp-Notebook {
    background-color: black;
}

</style>

In [ ]:
coverrepository = "<img src=""https://raw.githubusercontent.com/xlenore/ps2-covers/main/covers/"""
cover_dictionary = {}
ending = ".jpg>"
coverstring = "cover"
cover = {}

for x in range(len(gamelist)):
    cover[coverstring+str(x)] = widgets.HTML(coverrepository + id_dictionary['ID' + str(x)] + ending)

locals().update(cover)
coverlist = []

for x in range(len(gamelist)):
    coverlist.append(str(eval('cover' + str(x))).replace('HTML(value=\'', '').replace('>\')',' id=' + id_dictionary['ID' + str(x)] + '>'))

sequence = ''.join(coverlist)

l2 = widgets.HTML(sequence)
h2 = widgets.HTML('Event info')
d2 = Event(source=l2, watched_events=['click'])

def handle_event(event):
    h2.value = (f"{event['target']['id']}")
    for x in range(len(gamelist)):
        if h2.value == id_dictionary['ID' + str(x)]:
            os.system('python send.py -i #YOURPS4IPv4 -f ' + '\"' + str(gamelist[x]).replace(os.getcwd() + '\\','') + '\"')
        
d2.on_dom_event(handle_event)
display.display(l2)